In [1]:

'''import numpy as np
import seaborn as sns
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import pandas as pd
from HAZI05 import KNNClassifier'''

'import numpy as np\nimport seaborn as sns\nfrom typing import Tuple\nfrom scipy.stats import mode\nfrom sklearn.metrics import confusion_matrix\nimport pandas as pd\nfrom HAZI05 import KNNClassifier'

In [110]:


import numpy as np

import seaborn as sns

from typing import Tuple

from scipy.stats import mode

from sklearn.metrics import confusion_matrix
import pandas as pd


class KNNClassifier:

    @property
    def k_neighbors(self):  # 1

        return self.k

    def __init__(self, k: int, test_split_ratio: float) -> None:

        self.k = k
        self.test_split_ratio = test_split_ratio

        self.x_test, self.y_test = None, None

        self.x_train, self.y_train = None, None

        self.y_preds = None

    def train_test_split(self, features: pd.DataFrame, labels: pd.Series) -> None:  # 3

        test_size = int(features.shape[0] * self.test_split_ratio)

        train_size = features.shape[0] - test_size

        assert features.shape[0] == test_size + train_size, "Size mismatch!"

        self.x_train  = features.iloc[:train_size, :].reset_index(drop=True)
        self.y_train = labels.iloc[:train_size].reset_index(drop=True)

        self.x_test = features.iloc[train_size:train_size + test_size, :].reset_index(drop=True)
        self.y_test = labels.iloc[train_size:train_size + test_size].reset_index(drop=True)

    def euclidean(self, element_of_x: pd.DataFrame) -> pd.DataFrame:  # 4

        return np.sqrt(np.sum((self.x_train - element_of_x)**2, axis=1))


    def predict(self,x_test:pd.DataFrame) -> pd.Series: #5
        labels_pred = []
        for idx, x_test_element in x_test.iterrows():
            distances = self.euclidean(x_test_element)
            distances = pd.concat([distances, self.y_train], axis=1).sort_values(by=0)
            label_pred = mode(distances.iloc[:self.k,1],keepdims=False).mode
            labels_pred.append(label_pred)
        self.y_preds = pd.Series(labels_pred)

    def accuracy(self) -> float:  # 6
        #self.y_test, self.y_preds = self.y_test.align(self.y_preds, axis=1, copy=False)
        #self.y_test.columns.name = None
        true_positive = (self.y_test == self.y_preds).sum()

        return true_positive / self.y_test.shape[0] * 100

    def confusion_matrix(self):  # 7
        conf_matrix = confusion_matrix(self.y_test, self.y_preds)

        return conf_matrix

    def best_k(self):
        ret = tuple((1, -1))
        og_k = self.k
        for i in range(1, 21):
            self.k = i
            self.predict(self.x_test)
            acc = round(self.accuracy(), 2)
            if acc > ret[1]:
                ret = (self.k, acc)
        return ret

    @staticmethod
    def load_csv(csv_path: str) -> Tuple[pd.DataFrame, pd.Series]:  # 2
        df = pd.read_csv(csv_path)
        df = df.sample(frac=1, random_state=42, ignore_index=True)
        x_ = df.iloc[:, :-1]
        # pd.DataFrame(df['Outcome']) dupla [[]] = dataframet ad vissza series helyett
        y_ = df.iloc[:, -1] #dataframe/series

        return x_, y_


In [111]:
knn = KNNClassifier(9, 0.2)
x, y = knn.load_csv(r'C:\Users\darab\source\repos\BEVADAT\QYZF3M_BEVADAT2022232\HAZI\HAZI05\diabetes.csv')
knn.train_test_split(x, y)


In [112]:

knn.predict(knn.x_test)
knn.best_k()
#knn.y_test

#knn.accuracy()

(9, 75.16)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_n = KNeighborsRegressor(n_neighbors=5)
knn_n.fit(knn.x_train, knn.y_train)
# Make point predictions on the test set using the fit model.
predictions = knn_n.predict(knn.x_test)
pd.DataFrame(predictions)

,0
0,0.0
1,0.4
2,0.6
3,0.6
4,0.6
...,...
148,0.6
149,0.0
150,0.4
151,0.8


In [ ]:
s = pd.Series([1, 2, 3])
s.shape[0] * 100

300